In [ ]:
import os

directory = "clips/trailers/"

for file in os.listdir(directory):
    if file.endswith(".mp4"):
        print("Running file: " + file)
        !python -m dvt video-viz {directory + file}
        

Running file: Action_Trailer_Vishal,_Tamannaah_Hiphop_Tamizha_ Sundar_C_Official.mp4
usage: python3 -m dvt video-viz [-h] [--dirout DIROUT]
                                [--pipeline-level PIPELINE_LEVEL]
                                [--diff-cutoff DIFF_CUTOFF]
                                [--cut-min-length CUT_MIN_LENGTH]
                                [--frequency FREQUENCY]
                                [--path-to-faces PATH_TO_FACES]
                                [--path-to-audio PATH_TO_AUDIO]
                                [--path-to-subtitle PATH_TO_SUBTITLE]
                                finput
python3 -m dvt video-viz: error: unrecognized arguments: Sundar_C_Official.mp4
Running file: American_Factory_Official_Trailer_Netflix.mp4
Using TensorFlow backend.
^C
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.7/runpy.py", line 85, in _run_code
   

In [ ]:
import csv, json, os
from statistics import mode

directory = "dvt-output-data/data/"

with open("AllOutputs.csv", 'w') as outfile:
    writer = csv.writer(outfile, delimiter=",") 
    writer.writerow(["Video name", "Num shots", "Num shots with people", "Avg people per shot", "Most common shot length",
                            "Avg shot duration (in seconds)", "Objects detected", "People detected"])
    for file in os.listdir(directory):
        if file != "toc.json":
            with open(directory + file, 'r') as videojson:
                data = json.load(videojson)
                name = data["meta"][0]["input_filename"]
                num_shots = len(data["cut"])


                # Find most common shot length
                shot_lengths = []
                for l in data["length"]:
                    shot_lengths.append(l["shot_length"])
                try:
                    mode_shot_length = mode(shot_lengths)
                except:
                    mode_shot_length = ""


                # Counting people and objects
                checked_frames = []
                num_shots_peopled = 0
                objects = []
                people_per_shot = {}
                for obj in data["obj"]:
                    if obj["frame"] in checked_frames:
                        if obj["category"] == "person":
                            people_per_shot[obj["frame"]] += 1
                    else:
                        # Count frames with people
                        if obj["category"] == "person":
                            num_shots_peopled += 1
                            people_per_shot[obj["frame"]] = 1
                        else:
                            people_per_shot[obj["frame"]] = 0
                        checked_frames.append(obj["frame"])

                    # Add objects
                    if obj["category"] != "person" and obj["category"] not in objects:
                        objects.append(obj["category"])

                avg_people_per_shot = sum(people_per_shot.values()) / len(people_per_shot.values())

                # Faces detected
                people = []
                if "people" in data:
                    for person in data["people"]:
                        if person["person"] not in people:
                            people.append(person["person"])

                # Calculate average shot length in seconds
                shot_durations = []
                for shot in data["cut"]:
                    shot_durations.append(shot["frame_end"]-shot["frame_start"])
                avg_shot_duration = (sum(shot_durations)/len(shot_durations))/data["meta"][0]["fps"]


                writer.writerow([name, num_shots, num_shots_peopled, avg_people_per_shot, mode_shot_length,
                            avg_shot_duration, '|'.join(objects), '|'.join(people)])             
                